# CREATION DE LA DAFRAME FINALE support a notre M.L.

Fusion de fichiers csv 1 par table

## Attention pour fonctionner 2 fichiers sont indispensables :
- ../DATA/list_codecommune_leg2022_50inscrits.txt <br/>List de toutes les clefs necessaires
- ../DATA/EXPORT_CSV/final_TAR_YYY_leg2022com <br/>Fichier target des resultats aux legislatives 2022 qui initialise notre dataframe

### PROCEDURE de traitement de dataframe
### 1 Initialisation avec taget
### 2 Parcours des fichiers csv du repertoire
    TESTS / actions
    - A: commence par final
    - B: validation nb de lignes 
    - C: validation des entetes
    - D: validation des clefs (depuis list_codecommune_leg2022_50inscrits.txt)
    - E: recuperation de la liste des bons prefixes a traiter (file_name.split('_')[1:-1])
    - F: recuperation de la dataframe avec les colonnes qui disposent des prefixes souhaités
    - G: fusion (merge) sur la colonne codecommune
    
### 3 [NON TRAITE] : remplacement des Nan avec voisin le plus proche

### 4 [NON TRAITE] : Export CSV 

    

In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import pandas as pd
import numpy as np

## LISTER LES FICHIERS A TRAITER

In [2]:
# INPUT : Repertoire à parcourir
# OUTPUT : Dico [nom du fichier]: prefix des colonnes à traiter

def trouver_fichiers_csv(directory):
    fichiers_csv = {}
    files = os.listdir(directory)
    for file in files:
        # file_path = os.path.join(directory, file)
        file_name, file_extension = os.path.splitext(file)
        if file_extension == '.csv' :
            infos = file_name.split('_')
            if infos[0] =='final' and len(infos[1])==3 :
                fichiers_csv[file] = infos[1:-1]
    return fichiers_csv



In [3]:
path='../DATA/EXPORT_CSV/'
list_csv = trouver_fichiers_csv(path)
list_csv 

{'final_RSA_rsacommunes.csv': ['RSA'],
 'final_AGE_agesexcommunes.csv': ['AGE'],
 'final_BFL_basesfiscalescommunes.csv': ['BFL'],
 'final_GEO_correspondance-code-insee-code-postal.csv': ['GEO'],
 'final_KP0_capitalimmobilier.csv': ['KP0'],
 'final_KP1_capitalimmobiliercommunes.csv': ['KP1'],
 'final_PRO_proprietairescommunes.csv': ['PRO'],
 'final_EMP_empfoncommunes.csv': ['EMP'],
 'final_DIP_diplomescommunes.csv': ['DIP'],
 'final_CED_crimesdelits.csv': ['CED'],
 'final_CSP_cspcommunes.csv': ['CSP'],
 'final_TAR_YYY_leg2022comm.csv': ['TAR', 'YYY'],
 'final_TER_terrescommunes.csv': ['TER'],
 'final_NAT_naticommunes.csv': ['NAT']}

## FIXER LES INDEXES

In [4]:
# Specify the file path
file_path = '../DATA/list_codecommune_leg2022_50inscrits.txt'

# Open the text file and read lines into a list
with open(file_path, 'r', encoding='utf-8') as file:
    lines = [line.strip() for line in file.readlines()]

# Print or manipulate the list of lines as needed
print(lines[0:10], len(lines))

['01001', '01002', '01004', '01005', '01006', '01007', '01008', '01009', '01010', '01011'] 33830


# IMPORTS

In [5]:
# INPUT : 
#    nom du fichier
#    repertoire
#    liste des clefs (codecommunes)
#    colonnes entetes a conserver (codecommunes) pour tous sauf la target (on garde les 4)

# OUTPUT : dataframe du fichier csv controlé et filtré

def load_verif_df (file, path, listinfo, lines, cols_entete = ['codecommune'] ) :
    file_path = path + file
    print(f"\nTRAITEMENT : {file_path}")
    if os.path.exists(file_path):
        # IMPORT CSV
        df_from_csv = pd.read_csv(file_path, encoding='utf-8', sep=';', low_memory=False)
        print(f"   df_from_csv : {df_from_csv.shape}")
        # TEST DIM
        if len(lines) == len(df_from_csv) :
            # VALIDATION ENTETES
            cols = df_from_csv.columns
            if np.array_equal(cols[0:4],  ['dep', 'nomdep', 'codecommune', 'nomcommune']) : 
                # VALIDATION DES CLEFS
                if set(lines) == set(df_from_csv['codecommune'].tolist()):                                            
                    # LIST DES COLONNES A RECUP & FILTRE
                    cols_select = cols_entete + [col for col in cols[4:] if any(col.startswith(prefix) for prefix in listinfo)]
                    print(f"   > {len(cols_select)-1} COLONNES A TRAITER  {cols_select}")
                    return df_from_csv[cols_select]
                else : 
                    print(f"LISTE DES CLEFS NON RESPECTEE")
            else : 
                print(f"COLONNES ENTETES NON VALIDEES  {cols[0:4]}")
        else : 
            print(f"ERREUR NB. LIGNES : {len(lines)} != {len(df_from_csv)}")
    else:
        print(f"FICHIER {file_path} NON TROUVE")
    return pd.DataFrame()

### Initialisation

In [6]:
synthese = {}

### IMPOTER LA TARGET

(dico = TAR)

In [7]:
for dico_fil in list_csv : 
    if 'TAR' in list_csv[dico_fil] :
        df_final = load_verif_df (dico_fil, path, list_csv[dico_fil], lines, cols_entete = ['dep', 'nomdep', 'codecommune', 'nomcommune'])
        synthese[dico_fil] = df_final.shape[1]
        print (df_final.shape)



TRAITEMENT : ../DATA/EXPORT_CSV/final_TAR_YYY_leg2022comm.csv
   df_from_csv : (33830, 30)
   > 18 COLONNES A TRAITER  ['dep', 'nomdep', 'codecommune', 'nomcommune', 'TAR_inscrits', 'TAR_votants', 'TAR_exprimes', 'YYY_pvoixAUG', 'YYY_pvoixNUP', 'YYY_pvoixDVG', 'YYY_pvoixECO', 'YYY_pvoixREG', 'YYY_pvoixENS', 'YYY_pvoixUDI', 'YYY_pvoixLR', 'YYY_pvoixDVD', 'YYY_pvoixREC', 'YYY_pvoixRN', 'TAR_electeurs']
(33830, 19)


### FUSIONNER LES AUTRES FICHIERS DU DICO 
(hors TAR)

In [8]:
print(f"DIMENSIONS df_final EN ENTREE :  {df_final.shape}")
for dico_fil in list_csv : 
    if 'TAR' not in list_csv[dico_fil] :
        df = load_verif_df (dico_fil, path, list_csv[dico_fil], lines)
        synthese[dico_fil] = df.shape[1]-1
        df_final = pd.merge(df_final, df, on='codecommune', how='inner')
        print(f"DIMENSIONS df_final EN SORTIE :  {df_final.shape}")


DIMENSIONS df_final EN ENTREE :  (33830, 19)

TRAITEMENT : ../DATA/EXPORT_CSV/final_RSA_rsacommunes.csv
   df_from_csv : (0, 1)
ERREUR NB. LIGNES : 33830 != 0


KeyError: 'codecommune'

In [ ]:
synthese

In [ ]:
df_final['key']=df_final['codecommune']

In [ ]:
df_final.set_index('key', inplace=True)
df_final.head(2)

In [ ]:
index_suppr = ['13201',
 '13202',
 '13203',
 '13204',
 '13205',
 '13206',
 '13207',
 '13208',
 '13209',
 '13210',
 '13211',
 '13212',
 '13213',
 '13214',
 '13215',
 '13216',
 '69380',
 '69381',
 '69382',
 '69383',
 '69384',
 '69385',
 '69386',
 '69387',
 '69388',
 '69389',
 '75056']



In [ ]:
df_final = df_final[~df_final['codecommune'].isin(index_suppr)]
df_final.shape

In [ ]:
rc=''
cpte = -1
for i,c in enumerate (df_final) :
    cpte+=1
    if c[0:3] != rc and i>3 :
        print('\n>>>> ',rc,':  + ',cpte, ' = ',i)
        rc = c[0:3]
        cpte = 0
    print (c,end=' - ')
print('\n>>>> ',rc,':  + ',cpte, ' = ',i)

## TRAITER LES Nan RESTANTS

In [ ]:
df_final.isna().sum().sum()

In [ ]:
nb_lines = len(df_final)
for c in df_final :
    nb_nan = df_final[c].isna().sum()
    if nb_nan > 0 :
        print (c,nb_nan, round(nb_nan/nb_lines*100,2),'%', end = '| ')

In [ ]:
for c in df_final :
    print (c , type(c), end = '-')

In [ ]:
infos = df_final.iloc[:, 0:4]
X_tmp= df_final.iloc[:, 4:]
infos.head


In [ ]:
# X_tmp.astype(int)
for c in X_tmp :
    print (c, end ='-')
    X_tmp[c] = pd.to_numeric(X_tmp[c], errors='coerce')

#### X_tmp.describe()

In [ ]:
X_tmp['YYY_pvoixNUP'].isna().sum()

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
KNNI = KNNImputer(n_neighbors=5)

In [ ]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
df_finalC = pd.DataFrame(imputer.fit_transform(X_tmp),columns=X_tmp.columns)

In [ ]:
df_finalC.isna().sum().sum()

In [ ]:
df_finalC.iloc[0:10].shape

## EXPORTER

In [ ]:
path_export='../DATA/final_DATA.csv'
df_finalC.to_csv(path_export, sep=';', index=True)
print(f'DataFrame exported to: {path_export}')

In [ ]:
path_export='../DATA/final_4first_col.csv'
infos.to_csv(path_export, sep=';', index=True)
print(f'DataFrame exported to: {path_export}')